# Module 1 Homework

In [29]:
# Imports
from IPython.display import display
import numpy as np
import pandas as pd

# Finance Data Sources
import yfinance as yf
import pandas_datareader as pdr
import pandas_datareader.data as web

import time
from datetime import date

In [63]:
end = date.today()

start = date(year=end.year-5, month=end.month, day=end.day)
print(f"period for indexes: {start} to {end}")

period for indexes: 2019-04-21 to 2024-04-21


### Question 1: [Macro] Average growth of GDP in 2023
What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [81]:
def compute_average_growth(symbol, source, start, end):
    """
    Compute Average growth of GDP in 2023
    Args: 
        symbol: stock symbol
        source: fred
        start(date): start of period
        end(date): end of period
    Returns:
        rgdp_2023: (DataFrame): Dataframe containing required stock details for 2023
        av_growth: (float): Average growth of GDP in 2023
    """
    
    # Real Gross Domestic Product (GDPC1)
    rgdp = web.DataReader(symbol, source, start, end)

    # calculate the percentage change in GDP per capita over a 4-period interval (yearly and quarterly)
    rgdp['rgdp_us_yoy'] = rgdp.GDPC1/rgdp.GDPC1.shift(4)-1

    # Select period of interest
    rgdp_2023 = rgdp[(rgdp.index >= '2023-01-01') & (rgdp.index <= '2023-10-01')]

    av_growth = rgdp_2023['rgdp_us_yoy'].mean() * 100

    return rgdp_2023, av_growth

In [82]:
rgdp_2023, av_growth = compute_average_growth('GDPC1', 'fred', start, end)

In [83]:
display(rgdp_2023)

,GDPC1,rgdp_us_yoy
DATE,,
2023-01-01,22112.329,0.017179
2023-04-01,22225.350,0.023825
2023-07-01,22490.692,0.029269
2023-10-01,22679.255,0.031345


In [84]:
print('Average growth of GDP in 2023 (in %) is:', round(av_growth, 1))

Average growth of GDP in 2023 (in %) is: 2.5


### Question 2. [Macro] Inverse "Treasury Yield"
Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [92]:
dgs10 = web.DataReader('DGS10', 'fred', start, end)
dgs2 = web.DataReader('DGS2', 'fred', start, end)

In [93]:
dgs10.tail()

,DGS10
DATE,
2024-04-12,4.50
2024-04-15,4.63
2024-04-16,4.67
2024-04-17,4.59
2024-04-18,4.64


In [94]:
dgs2.tail()

,DGS2
DATE,
2024-04-12,4.88
2024-04-15,4.93
2024-04-16,4.97
2024-04-17,4.93
2024-04-18,4.98


In [96]:
merged_df = pd.concat([dgs10, dgs2], axis=1)
merged_df.tail()

,DGS10,DGS2
DATE,,
2024-04-12,4.50,4.88
2024-04-15,4.63,4.93
2024-04-16,4.67,4.97
2024-04-17,4.59,4.93
2024-04-18,4.64,4.98


In [97]:
merged_df['dgs10-dgs2'] = merged_df['DGS10'] - merged_df['DGS2']
merged_df.tail()

,DGS10,DGS2,dgs10-dgs2
DATE,,,
2024-04-12,4.50,4.88,-0.38
2024-04-15,4.63,4.93,-0.30
2024-04-16,4.67,4.97,-0.30
2024-04-17,4.59,4.93,-0.34
2024-04-18,4.64,4.98,-0.34


In [98]:
merged_df_trunc = merged_df[(merged_df.index >= '2000-01-01')]
inverse_treasury_yield = merged_df_trunc['dgs10-dgs2'].min().round(1)
print(f'The Inverse Treasury Yield of dgs10-dgs2 is {inverse_treasury_yield}')

The Inverse Treasury Yield of dgs10-dgs2 is -1.1


### Question 3. [Index] Which Index is better recently?
Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [111]:
def compute_growth(tickers, start, end, interval):
    """
    Compute Average growth of GDP in 2023
    Args: 
        tickers: list of stock symbol
        interval(int): frequency 
        start(date): start of period
        end(date): end of period
    Returns:
        stock_data (Dataframe): DataFrame containing historical stock data for each stock
    """
    
    stock_data = {}
    stock_growth = {}
    
    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        
        start_price = data.loc[start_date]['Close']
        end_price = data.loc[str(data.index[-1].strftime('%Y-%m-%d'))]['Close']

        growth = ((end_price - start_price) / start_price * 100).round()

        stock_data[ticker] = data
        stock_growth[ticker] = growth
        
    return stock_data, stock_growth

In [112]:
start_date = "2019-04-09"
end_date = "2024-04-09"
tickers = ["^GSPC", "^MXX"]
stock_data, stock_growth = compute_growth(tickers, start_date, end_date, interval='1d')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [113]:
for ticker, data in stock_data.items():
    print(f"DataFrame: {ticker}")
    display(data.tail())

DataFrame: ^GSPC


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-02,5204.290039,5208.339844,5184.049805,5205.810059,5205.810059,3886590000
2024-04-03,5194.370117,5228.750000,5194.370117,5211.490234,5211.490234,3703250000
2024-04-04,5244.049805,5256.589844,5146.060059,5147.209961,5147.209961,4075680000
2024-04-05,5158.950195,5222.180176,5157.209961,5204.339844,5204.339844,3386780000
2024-04-08,5211.370117,5219.569824,5197.350098,5202.390137,5202.390137,3278180000


DataFrame: ^MXX


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-02,57593.621094,57830.878906,57235.589844,57581.808594,57581.808594,180753600
2024-04-03,57547.191406,58086.421875,57300.109375,57503.390625,57503.390625,189285300
2024-04-04,57539.468750,58219.500000,57514.179688,57882.761719,57882.761719,184739700
2024-04-05,57805.191406,58227.839844,57678.609375,58092.441406,58092.441406,212252300
2024-04-08,58021.660156,58192.320312,57749.441406,57989.941406,57989.941406,154641800


In [115]:
for ticker, growth in stock_growth.items():
    print(f"5-Year Growth for {ticker} is {growth}")

5-Year Growth for ^GSPC is 81.0
5-Year Growth for ^MXX is 28.0


### Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks
Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [116]:
# get actions, incl. dividends - as a dataFrame
def get_stock_data(stock_symbols, start_date, end_date, interval):
    """
    Compute stock data
    Args: 
        stock_symbols: list of stock symbols
        interval(int): frequency 
        start(date): start of period
        end(date): end of period
    Returns:
        stock_data_dict (Dataframe): DataFrame containing historical stock data for each stock
    """
    
    stock_data_dict = {}
    
    for symbol in stock_symbols:
        data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        stock_data_dict[symbol] = data

    return stock_data_dict

In [117]:
stock_symbols = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]
start_date = '2023-01-01'
end_date = '2023-12-31'

In [130]:
stock_data_dict = get_stock_data(stock_symbols, start_date, end_date, interval='1d')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [164]:
for symbol, data in stock_data_dict.items():
    print(data.loc[str(data.index[-1].strftime('%Y-%m-%d'))]['Adj Close'])

32.87778854370117
356.6600036621094
192.28463745117188
375.34588623046875
140.92999267578125
168.07713317871094


In [165]:
for symbol, data in stock_data_dict.items():
    print(f"DataFrame: {symbol}")
    display(data.tail())

DataFrame: 2222.SR


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-24,32.950001,33.099998,32.849998,33.000000,32.828049,12165552
2023-12-25,33.000000,33.099998,32.849998,33.099998,32.927525,15869070
2023-12-26,33.049999,33.099998,32.900002,33.049999,32.877789,14598967
2023-12-27,33.099998,33.250000,33.000000,33.150002,32.977268,14815683
2023-12-28,33.150002,33.250000,32.950001,33.049999,32.877789,12720450


DataFrame: BRK-B


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,356.500000,358.260010,355.410004,356.470001,356.470001,2332200
2023-12-26,356.899994,357.769989,356.079987,356.829987,356.829987,1964400
2023-12-27,355.929993,357.079987,355.500000,356.950012,356.950012,2478500
2023-12-28,357.480011,358.679993,356.730011,357.570007,357.570007,2740600
2023-12-29,357.559998,357.859985,355.309998,356.660004,356.660004,3269400


DataFrame: AAPL


...

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,373.679993,375.179993,372.709991,374.579987,373.888580,17091100
2023-12-26,375.000000,376.940002,373.500000,374.660004,373.968445,12673100
2023-12-27,373.690002,375.059998,372.809998,374.070007,373.379547,14905400
2023-12-28,375.369995,376.459991,374.160004,375.279999,374.587280,14327000
2023-12-29,376.000000,377.160004,373.480011,376.040009,375.345886,18723000


DataFrame: GOOG


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,142.130005,143.250000,142.054993,142.720001,142.720001,18494700
2023-12-26,142.979996,143.945007,142.500000,142.820007,142.820007,11170100
2023-12-27,142.830002,143.320007,141.050995,141.440002,141.440002,17288400
2023-12-28,141.850006,142.270004,140.828003,141.279999,141.279999,12192500
2023-12-29,140.679993,141.434998,139.899994,140.929993,140.929993,14872700


DataFrame: JPM


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-12-22,167.500000,168.229996,167.229996,167.399994,165.409210,6574900
2023-12-26,167.460007,168.770004,167.179993,168.389999,166.387451,4683600
2023-12-27,167.839996,169.470001,167.580002,169.399994,167.385437,6428600
2023-12-28,169.350006,170.660004,169.000000,170.300003,168.274750,6320100
2023-12-29,170.000000,170.690002,169.630005,170.100006,168.077133,6431800


In [150]:
# Compute drawdown for each stock

def compute_drawdown(stock_data_dict):
    """
    Compute drawdown for a given stock
    Args: 
        stock_data_dict (DataFrame): Historical stock data containing 'Date' and 'Adj close' columns
    Returns:
        Drawdown: (float): Drawdown of the stock
    """
    drawdowns = {}
    
    for symbol, data in stock_data_dict.items():
    
        max_price = data['Adj Close'].max()
    
        # Calculate the min_price to 2d.p.
        min_price = data['Adj Close'].min()  
    
        # Calculate Drawdown of stock
        drawdown = ((max_price - min_price) / max_price).round(2)

        drawdowns[symbol] = drawdown

    return drawdowns

In [134]:
drawdowns = compute_drawdown(stock_data_dict)

# Print drawdown for each stock
for symbol, drawdown in drawdowns.items():
    print(f'Drawdown of {symbol}: {drawdown}')

Drawdown of 2222.SR: 0.21
Drawdown of BRK-B: 0.21
Drawdown of AAPL: 0.37
Drawdown of MSFT: 0.42
Drawdown of GOOG: 0.39
Drawdown of JPM: 0.28


#### Importance of Drawdown
The expression (max - min) / max price of a stock over a period of time is commonly referred to as the <b>drawdown</b>

Drawdown measures the peak-to-trough decline in the value of a stock or investment over a specific time period. It represents the percentage decline from the highest point (peak) to the lowest point (trough) in the investment's value.

It is an important metric in stock analysis

It serves as a valuable tool for investors to gauge risk, evaluate performance, and make well-informed investment decisions in the stock market.

### Question 5. [Stocks] Dividend Yield
Find the largest dividend yield for the same set of stocks

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximm value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [168]:
# get actions, incl. dividends - as a dataFrame
def get_dividend_yield(stock_symbols, start_date, end_date, interval):

    """
    Compute closing price and total dividends for stocks in the stock_symbol list
    Args: 
        stock_symbols: list of stock symbols
        interval(int): frequency 
        start_date(date): start of period
        end_date(date): end of period
    Returns:
        closing_price_dict (Dataframe): DataFrame containing closing price for each stock
        total_dividends_dict (Dataframe): DataFrame containing sum of dividends for each stock for the stated period
    """
    
    closing_price_dict = {}
    total_dividends_dict = {}
    
    for symbol in stock_symbols:
        data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        closing_price = data.loc[str(data.index[-1].strftime('%Y-%m-%d'))]['Adj Close']
        closing_price_dict[symbol] = closing_price

        div_data = yf.Ticker(symbol)
        dividends = div_data.dividends.loc[start_date:end_date]
        total_dividends = dividends.values.sum()
        total_dividends_dict[symbol] = total_dividends

    return closing_price_dict, total_dividends_dict

In [170]:
closing_price_dict, total_dividends_dict = get_dividend_yield(stock_symbols, start_date, end_date, interval='1d')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [207]:
# compute Dividends yield
def compute_dividend_yield(total_dividends_dict, closing_price_dict):
    """
    Compute dividend yiled for stocks in the stock_symbol list
    Args: 
        closing_price_dict (Dataframe): DataFrame containing closing price for each stock
        total_dividends_dict (Dataframe): DataFrame containing sum of dividends for each stock for the stated period

    Return:
        Dividend_yield_dict (DataFrame): DataFrame containing Dividend yield for each stock for the stated period

    """
    dividend_yield_dict = {}
    for symbol, total_dividends in total_dividends_dict.items():
        closing_price = closing_price_dict[symbol]
        dividend_yield = (total_dividends / closing_price) * 100
        dividend_yield_dict[symbol] = round(dividend_yield, 1)

    return dividend_yield_dict

In [208]:
dividend_yield_dict = compute_dividend_yield(total_dividends_dict, closing_price_dict)

In [209]:
for symbol, dividend_yield in dividend_yield_dict.items():
    print(f"\nDividend_yield for {symbol}: {dividend_yield}\n")


Dividend_yield for 2222.SR: 2.8


Dividend_yield for BRK-B: 0.0


Dividend_yield for AAPL: 0.5


Dividend_yield for MSFT: 0.7


Dividend_yield for GOOG: 0.0


Dividend_yield for JPM: 2.4



### Question 6. [Exploratory] Investigate new metrics
Free text answer

Download and explore a few additional metrics or time series that might be valuable for your project and write down why (briefly).

Here are some additional metrics to consider when investing in stocks:

<b>Price-to-Earnings (P/E) Ratio:</b> Measures the valuation of a company's stock by comparing its current price to its earnings per share (EPS). A lower P/E ratio may indicate that a stock is undervalued, while a higher P/E ratio may indicate that it is overvalued.

<b>Price-to-Book (P/B) Ratio:</b> Compares a company's market value to its book value, calculated by dividing the current share price by the book value per share. It provides insights into whether a stock is trading at a discount or premium relative to its book value.

<b>Price-to-Sales (P/S) Ratio:</b> Compares a company's market capitalization to its revenue. It indicates how much investors are willing to pay for each dollar of the company's sales and is useful for evaluating companies with low or negative earnings.

<b>Return on Equity (ROE):</b> Measures a company's profitability by calculating how much profit it generates with the money shareholders have invested. It is calculated as net income divided by shareholders' equity.Debt-to-Equity Ratio: Compares a company's total debt to its shareholders' equity. It indicates the proportion of debt financing relative to equity financing and provides insights into a company's financial leverage and risk.Earnings Growth: Examines the rate at which a company's earnings are growing over time. Consistent and sustainable earnings growth is often a sign of a healthy and successful company.

<b>Dividend Payout Ratio:</b> Measures the percentage of earnings that a company pays out to shareholders in the form of dividends. It indicates how much of the company's earnings are being distributed to shareholders versus being retained for reinvestment.Beta: Measures a stock's volatility relative to the overall market. A beta greater than 1 indicates that the stock is more volatile than the market, while a beta less than 1 indicates lower volatility.Volatility: Measures the degree of variation in a stock's trading price over time. High volatility stocks tend to have wider price swings, while low volatility stocks have more stable prices.

<b>Profit Margin:</b> Measures a company's profitability by comparing its net income to its revenue. It indicates the percentage of revenue that translates into profit after all expenses are accounted for.

These metrics are important for investors and analysts because they provide valuable insights into various aspects of a company's financial health, performance and valuation.

We will compute these metrics for the six stocks we have been analysing

In [197]:
def compute_metrics(stock_symbols, start_date, end_date):
    """
    Compute various metrics for a list of stock symbols for a specific period.

    Args:
    stock_symbols (list): List of stock symbols (tickers).
    start_date (str): Start date in 'YYYY-MM-DD' format.
    end_date (str): End date in 'YYYY-MM-DD' format.

    Returns:
    metrics_df (DataFrame): DataFrame containing computed metrics for each stock.
    """
    # Initialize an empty DataFrame to store computed metrics
    metrics_df = pd.DataFrame(index=stock_symbols)

    # Loop through each stock symbol
    for symbol in stock_symbols:
        try:
            # Create a Ticker object for the stock symbol
            ticker = yf.Ticker(symbol)

            # Get summary data
            summary_data = ticker.info

            # Calculate P/E ratio
            pe_ratio = summary_data.get('trailingPE', None)
            metrics_df.loc[symbol, 'P/E Ratio'] = pe_ratio

            # Calculate P/B ratio
            pb_ratio = summary_data.get('priceToBook', None)
            metrics_df.loc[symbol, 'P/B Ratio'] = pb_ratio

            # Calculate P/S ratio
            ps_ratio = summary_data.get('priceToSalesTrailing12Months', None)
            metrics_df.loc[symbol, 'P/S Ratio'] = ps_ratio

            # Calculate ROE
            roe = summary_data.get('returnOnEquity', None)
            metrics_df.loc[symbol, 'ROE'] = roe

            # Calculate Debt-to-Equity ratio
            debt_to_equity = summary_data.get('debtToEquity', None)
            metrics_df.loc[symbol, 'Debt-to-Equity'] = debt_to_equity


            # Calculate Volatility (standard deviation of daily returns)
            historical_data = ticker.history(start=start_date, end=end_date)
            daily_returns = historical_data['Close'].pct_change().dropna()
            volatility = daily_returns.std()
            metrics_df.loc[symbol, 'Volatility'] = volatility

            # Calculate Profit Margin
            profit_margin = summary_data.get('profitMargins', None)
            metrics_df.loc[symbol, 'Profit Margin'] = profit_margin

        except Exception as e:
            print(f"Error computing metrics for {symbol}: {e}")

    return metrics_df

In [198]:
# Call the function to compute metrics for the list of stocks
metrics_df = compute_metrics(stock_symbols, start_date, end_date)

# Display the DataFrame containing computed metrics for each stock
display(metrics_df)

,P/E Ratio,P/B Ratio,P/S Ratio,ROE,Debt-to-Equity,Volatility,Profit Margin
2222.SR,15.989305,4.713858,3.897123,0.26725,16.704,0.008808,0.24389
BRK-B,9.148149,0.001041,2.402315,0.18461,23.402,0.008610,0.26400
AAPL,25.660965,34.425200,6.605843,1.54269,145.803,0.012570,0.26163
MSFT,36.086796,12.449157,13.031014,0.39174,46.736,0.015824,0.36269
GOOG,26.894646,6.846942,6.263166,0.27356,10.540,0.019283,0.24007
JPM,11.213036,1.739424,3.557918,0.15741,NaN,0.013088,0.33569


### Question 7. [Exploratory] Time-driven strategy description around earnings releases
Free text answer

Explore earning dates for the whole month of April - e.g. using YahooFinance earnings calendar (https://finance.yahoo.com/calendar/earnings?from=2024-04-21&to=2024-04-27&day=2024-04-23). Compare with the previous closed earnings (e.g., recent dates with full data https://finance.yahoo.com/calendar/earnings?from=2024-04-07&to=2024-04-13&day=2024-04-08).

Describe an analytical strategy/idea (you're not required to implement it) to select a subset companies of interest based on the future events data.

#### Analytical Strategy/Idea:
Here are some potential strategies for selecting a subset of companies of interest:

<b>Volatility-Based Approach:</b>

Identify companies that historically exhibit significant price volatility around earnings announcements.
Look for stocks with large price swings (both positive and negative) following earnings releases.
Consider companies with a history of beating or missing earnings estimates by a wide margin.
This approach could be useful for short-term traders looking to capitalize on price movements.

<b>Sector or Industry Focus:</b>
Focus on specific sectors or industries that are expected to perform well or have favorable market conditions.
For example, if the technology sector is expected to outperform, consider companies within that sector.
Conversely, avoid sectors facing headwinds or regulatory challenges.

<b>Quality Metrics:</b>
Evaluate companies based on fundamental metrics such as revenue growth, profit margins, and return on equity.
Look for companies with consistent positive earnings surprises and strong financial health.
Consider using ratios like the price-to-earnings (P/E) ratio to assess valuation.

<b>Event-Driven Trading:</b>
Consider event-driven trading strategies, where you take positions before or after specific events (e.g., earnings announcements).
Look for companies with upcoming catalysts (e.g., product launches, mergers, or acquisitions) in addition to earnings releases.
Sentiment Analysis: